# Démonstration du système RAG Multimodal avec CLIP

Ce notebook montre comment utiliser le système de Retrieval Augmented Generation (RAG) multimodal qui permet de combiner des requêtes sur des textes et des images grâce aux embeddings CLIP.

Le système permet :
- De représenter les textes et les images dans un même espace vectoriel
- D'effectuer des requêtes textuelles qui peuvent retourner des images pertinentes
- D'effectuer des requêtes avec des images qui peuvent retourner du texte pertinent
- D'analyser des documents PDF en extrayant à la fois le texte et les images

## 1. Installation des dépendances

Assurez-vous d'avoir installé toutes les dépendances nécessaires :

In [ ]:
!pip install langchain langchain-core langchain-community langchain-ollama
!pip install torch transformers clip pymupdf pillow chromadb
!pip install matplotlib tqdm numpy requests

## 2. Importation des modules nécessaires

Importons les classes du système RAG multimodal ainsi que d'autres bibliothèques utiles :

In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO

# Importer notre système RAG multimodal
from multimodal_rag import MultimodalEmbedder, MultimodalVectorStore, MultimodalRAG

## 3. Vérification d'Ollama

Assurons-nous qu'Ollama est en cours d'exécution et que les modèles nécessaires sont disponibles :

In [ ]:
# Vérifier si Ollama est en cours d'exécution
try:
    response = requests.get("http://localhost:11434/api/tags")
    if response.status_code == 200:
        print("✅ Ollama est en cours d'exécution")
        # Afficher les modèles disponibles
        models = response.json()["models"]
        print(f"Modèles disponibles: {', '.join([m['name'] for m in models])}")
    else:
        print("❌ Ollama est en cours d'exécution mais a retourné une erreur")
except Exception as e:
    print(f"❌ Ollama n'est pas en cours d'exécution ou n'est pas accessible: {e}")
    print("Lancez Ollama avec la commande: ollama serve")

## 4. Initialisation du système RAG multimodal

Initialisons notre système RAG multimodal avec le modèle de notre choix :

In [ ]:
# Modèle Ollama à utiliser - vous pouvez changer pour un autre modèle disponible
LLM_MODEL = "qwen2.5:3b"  # ou "llava:7b", "llama2:7b", etc.

# Dossier où seront stockés les embeddings
DB_PATH = "demo_chroma_db"

# Initialiser le système RAG
print(f"Initialisation du système RAG multimodal avec le modèle {LLM_MODEL}...")
rag_system = MultimodalRAG(
    llm_name=LLM_MODEL,
    collection_name="demo_collection",
    persist_directory=DB_PATH
)

## 5. Exploration des embeddings CLIP

CLIP (Contrastive Language-Image Pre-training) est un modèle qui aligne les représentations de texte et d'image dans un même espace vectoriel. Explorons comment cela fonctionne :

In [ ]:
# Initialiser l'embedder CLIP
embedder = MultimodalEmbedder()

# Exemples de textes
texts = [
    "Un chien qui court dans un parc",
    "Un chat qui dort sur un canapé",
    "Une voiture rouge garée devant une maison",
    "Un coucher de soleil sur la plage",
    "Un ordinateur portable sur un bureau"
]

# Générer des embeddings pour ces textes
text_embeddings = embedder.embed_text(texts)
print(f"Forme des embeddings de texte: {text_embeddings.shape}")

In [ ]:
# Télécharger quelques images d'exemple
image_urls = [
    "https://images.unsplash.com/photo-1543466835-00a7907e9de1?w=500",  # Chien
    "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=500",  # Chat
    "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=500",  # Voiture
    "https://images.unsplash.com/photo-1493514789931-586cb221d7a7?w=500",  # Coucher de soleil
    "https://images.unsplash.com/photo-1498050108023-c5249f4df085?w=500"   # Ordinateur
]

# Télécharger et afficher les images
images = []
plt.figure(figsize=(15, 10))
for i, url in enumerate(image_urls):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    images.append(img)
    
    plt.subplot(1, 5, i+1)
    plt.imshow(img)
    plt.title(f"Image {i+1}")
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Générer des embeddings pour ces images
image_embeddings = embedder.embed_image(images)
print(f"Forme des embeddings d'image: {image_embeddings.shape}")

### Visualisation de la similarité texte-image

Un des avantages de CLIP est qu'il place les textes et les images dans le même espace vectoriel, permettant de calculer des similarités entre eux :

In [ ]:
# Calculer la similarité cosinus entre les textes et les images
similarity_matrix = np.dot(text_embeddings, image_embeddings.T)

# Afficher la matrice de similarité sous forme de heatmap
plt.figure(figsize=(10, 8))
plt.imshow(similarity_matrix, cmap='viridis')
plt.colorbar(label='Similarité cosinus')
plt.xticks(range(len(image_urls)), [f"Image {i+1}" for i in range(len(image_urls))], rotation=45)
plt.yticks(range(len(texts)), [t[:30] + '...' if len(t) > 30 else t for t in texts])
plt.title('Similarité entre textes et images')
plt.tight_layout()
plt.show()

In [ ]:
# Afficher les meilleures correspondances
for i, text in enumerate(texts):
    best_image_idx = np.argmax(similarity_matrix[i])
    print(f"Meilleure image pour '{text}': Image {best_image_idx+1} (similarité: {similarity_matrix[i, best_image_idx]:.4f})")

## 6. Création d'une base de connaissances multimodale

Créons un dossier pour stocker nos données et ajoutons des documents à notre base de connaissances :

In [ ]:
# Créer un dossier pour stocker les fichiers téléchargés
os.makedirs('data_demo', exist_ok=True)

# Télécharger et sauvegarder les images
image_paths = []
for i, url in enumerate(image_urls):
    response = requests.get(url)
    image_path = f"data_demo/image_{i+1}.jpg"
    with open(image_path, 'wb') as f:
        f.write(response.content)
    image_paths.append(image_path)
    print(f"Image sauvegardée: {image_path}")

In [ ]:
# Créer quelques fichiers texte d'exemple
text_contents = [
    "Les chiens sont des animaux domestiques populaires. Ils sont connus pour leur loyauté et leur amitié. Les chiens peuvent être dressés pour effectuer diverses tâches, comme la garde, l'assistance aux personnes handicapées ou la recherche et le sauvetage.",
    "Les chats sont des animaux de compagnie indépendants et souvent affectueux. Ils sont connus pour leur agilité et leur capacité à chasser les souris. Les chats passent une grande partie de leur journée à dormir, environ 12 à 16 heures par jour.",
    "Les voitures sont des véhicules à moteur conçus pour le transport de personnes. Elles ont généralement quatre roues et peuvent accueillir entre une et huit personnes. Les voitures modernes sont équipées de nombreuses fonctionnalités de sécurité et de confort.",
    "Les couchers de soleil sont un phénomène naturel qui se produit lorsque le Soleil disparaît sous l'horizon. Ils sont souvent caractérisés par des couleurs spectaculaires dans le ciel, allant du jaune et orange au rouge et violet. Les plus beaux couchers de soleil se produisent souvent près de l'eau ou dans des zones avec peu de pollution lumineuse.",
    "Les ordinateurs portables sont des ordinateurs personnels compacts et facilement transportables. Ils combinent dans un seul appareil un écran, un clavier, un dispositif de pointage et une batterie. Les ordinateurs portables modernes offrent des performances comparables à celles des ordinateurs de bureau."
]

text_paths = []
for i, content in enumerate(text_contents):
    text_path = f"data_demo/document_{i+1}.txt"
    with open(text_path, 'w', encoding='utf-8') as f:
        f.write(content)
    text_paths.append(text_path)
    print(f"Document texte créé: {text_path}")

Maintenant, ajoutons ces documents à notre système RAG :

In [ ]:
# Ajouter les images avec des descriptions
for i, path in enumerate(image_paths):
    description = texts[i]  # Utiliser les mêmes textes que précédemment comme descriptions
    print(f"Ajout de l'image {i+1} avec description: '{description[:30]}...'")
    rag_system.add_document(path, document_type="image", description=description)
    time.sleep(0.5)  # Pause pour éviter de surcharger le système

# Ajouter les textes
for i, path in enumerate(text_paths):
    print(f"Ajout du document texte {i+1}")
    rag_system.add_document(path, document_type="text")
    time.sleep(0.5)  # Pause pour éviter de surcharger le système

print("\nTous les documents ont été ajoutés au système RAG!")

## 7. Interrogation du système RAG multimodal

Maintenant que notre base de connaissances est prête, essayons différentes requêtes pour voir comment le système répond :

### 7.1. Requête textuelle simple

In [ ]:
query = "Que font les chats pendant la journée?"
print(f"Requête: '{query}'\n")

result = rag_system.query(query)

print("Réponse:")
print("=" * 50)
print(result["answer"])
print("=" * 50)

print("\nSources utilisées:")
for i, source in enumerate(result["sources"]):
    print(f"Source {i+1} ({source['type']}): {source['metadata'].get('source', 'inconnue')}")
    print(f"Similarité: {source['similarity']:.4f}")
    print("-" * 40)

### 7.2. Requête conceptuelle (peut retourner des images ou du texte)

In [ ]:
query = "Parle-moi des animaux de compagnie"
print(f"Requête: '{query}'\n")

result = rag_system.query(query)

print("Réponse:")
print("=" * 50)
print(result["answer"])
print("=" * 50)

print("\nSources utilisées:")
for i, source in enumerate(result["sources"]):
    print(f"Source {i+1} ({source['type']}): {source['metadata'].get('source', 'inconnue')}")
    print(f"Similarité: {source['similarity']:.4f}")
    # Si c'est une image, l'afficher
    if source["type"] == "image" and "path" in source:
        img = Image.open(source["path"])
        plt.figure(figsize=(5, 5))
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Image source {i+1}")
        plt.show()
    print("-" * 40)

### 7.3. Requête avec une image

Maintenant, essayons d'interroger le système avec une image comme entrée :

In [ ]:
# Sélectionner une image pour la requête (par exemple, l'image du chat)
query_image_path = image_paths[1]

# Afficher l'image de requête
query_image = Image.open(query_image_path)
plt.figure(figsize=(5, 5))
plt.imshow(query_image)
plt.title("Image de requête")
plt.axis('off')
plt.show()

# Faire la requête avec l'image
print("Requête: Description de cette image\n")
result = rag_system.query(query_image_path)

print("Réponse:")
print("=" * 50)
print(result["answer"])
print("=" * 50)

print("\nSources utilisées:")
for i, source in enumerate(result["sources"]):
    print(f"Source {i+1} ({source['type']}): {source['metadata'].get('source', 'inconnue')}")
    print(f"Similarité: {source['similarity']:.4f}")
    print("-" * 40)

### 7.4. Combinaison texte + image avec une nouvelle image

Essayons avec une nouvelle image qui n'est pas dans notre base de connaissances :

In [ ]:
# Télécharger une nouvelle image qui n'est pas dans notre base
new_image_url = "https://images.unsplash.com/photo-1583795128727-6ec3642408f8?w=500"  # Un autre chien
response = requests.get(new_image_url)
new_image_path = "data_demo/new_dog.jpg"
with open(new_image_path, 'wb') as f:
    f.write(response.content)

# Afficher l'image
new_image = Image.open(new_image_path)
plt.figure(figsize=(5, 5))
plt.imshow(new_image)
plt.title("Nouvelle image (pas dans la base)")
plt.axis('off')
plt.show()

# Requête spécifique sur cette image
query = "Quelle race de chien est sur cette image et quelles sont les caractéristiques de cette race?"
print(f"Requête avec image: '{query}'\n")

# Faire la requête en ajoutant temporairement l'image à la base
rag_system.add_document(new_image_path, document_type="image", description="Nouvelle image d'un chien")
result = rag_system.query(query)

print("Réponse:")
print("=" * 50)
print(result["answer"])
print("=" * 50)

## 8. Exemple d'utilisation avec un document PDF

Pour montrer comment le système gère les PDF, on peut créer un PDF simple et l'analyser :

In [ ]:
# Pour créer un PDF d'exemple, nous allons utiliser fpdf (installer si nécessaire)
try:
    from fpdf import FPDF
except ImportError:
    !pip install fpdf
    from fpdf import FPDF

# Créer un PDF simple avec du texte et une image
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)

# Ajouter du texte
pdf.cell(200, 10, txt="Documentation sur les animaux domestiques", ln=True, align='C')
pdf.ln(10)
pdf.multi_cell(0, 10, txt="Les chiens et les chats sont les animaux de compagnie les plus populaires dans le monde. Ils offrent de la compagnie et peuvent améliorer la santé mentale de leurs propriétaires.")
pdf.ln(5)
pdf.multi_cell(0, 10, txt="Les chiens requièrent plus d'attention et d'exercice que les chats. Ils doivent être promenés régulièrement et ont besoin d'interactions sociales.")
pdf.ln(5)
pdf.multi_cell(0, 10, txt="Les chats sont plus indépendants mais nécessitent tout de même des soins attentifs. Ils aiment avoir des endroits calmes pour se reposer et dormir.")

# Ajouter une image
pdf.ln(10)
pdf.cell(200, 10, txt="Images d'exemples :", ln=True)
pdf.image(image_paths[0], x=30, y=100, w=70)  # Chien
pdf.image(image_paths[1], x=110, y=100, w=70)  # Chat

# Enregistrer le PDF
pdf_path = "data_demo/animaux_domestiques.pdf"
pdf.output(pdf_path)
print(f"PDF créé: {pdf_path}")

In [ ]:
# Ajouter le PDF à notre système RAG
print("Ajout du PDF au système RAG...")
pdf_ids = rag_system.add_document(pdf_path)
print(f"PDF ajouté avec succès! ({len(pdf_ids)} éléments indexés)")

In [ ]:
# Interroger notre système sur le contenu du PDF
query = "Quelle est la différence entre les chiens et les chats ?"
print(f"Requête: '{query}'\n")

result = rag_system.query(query)

print("Réponse:")
print("=" * 50)
print(result["answer"])
print("=" * 50)

print("\nSources utilisées:")
for i, source in enumerate(result["sources"]):
    source_type = source["type"]
    source_info = source["metadata"].get("source", "inconnue")
    page_info = f" (page {source['metadata']['page']})" if "page" in source["metadata"] else ""
    similarity = source["similarity"]
    
    print(f"Source {i+1} ({source_type}){page_info}: {source_info}")
    print(f"Similarité: {similarity:.4f}")
    
    # Si c'est une image extraite du PDF, l'afficher
    if source_type == "image" and "path" in source and "pdf_image" in source["metadata"].get("type", ""):
        img = Image.open(source["path"])
        plt.figure(figsize=(5, 5))
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Image extraite du PDF (page {source['metadata']['page']})")
        plt.show()
    
    print("-" * 40)

## 9. Conclusion

Ce notebook a démontré comment le système RAG multimodal peut être utilisé pour combiner des informations textuelles et visuelles dans un même système de question-réponse. Les avantages principaux sont :

1. **Unification des modalités** : Un seul système peut traiter à la fois du texte et des images.
2. **Requêtes cross-modales** : Une requête textuelle peut retourner des images pertinentes et vice versa.
3. **Flexibilité** : Le système peut être utilisé pour diverses applications comme la documentation technique, les catalogues de produits, l'analyse de PDFs, etc.
4. **Extensibilité** : D'autres types de documents peuvent être facilement intégrés.

Pour aller plus loin, il serait possible d'étendre ce système pour prendre en charge d'autres modalités comme l'audio ou la vidéo, ou d'améliorer les performances en utilisant des modèles d'embedding plus récents.